# Import packages

In [1]:
import argparse
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
from joblib import Parallel, delayed
import psutil
import multiprocessing
import time

# define helper functions

In [2]:
def print_memory_usage():
    print ("memory log:")
    process = psutil.Process(os.getpid())
    print("%5.2f GB (RSS)" % (process.memory_info().rss / 2**30))
    print("%5.2f GB (VMS)" % (process.memory_info().vms / 2**30))
    print("%5.2f GB (Used)" % (psutil.virtual_memory().used / 2**30))
    print("%5.2f GB (Available)" % (psutil.virtual_memory().available / 2**30))
    print("%5.2f GB (Total)" % (psutil.virtual_memory().total / 2**30))
    
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)
def applyParallel_list(pool,data, func, kwargs):
    data_split = np.array_split(data,partitions)
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    return data

# set parameters

In [25]:
cores = 5#cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want
threads=2
print ("number of cores "+str(partitions))

sample_rate = 1
raduis = 14 # mi

effective_days=300
max_effective_days = 21*24*60

number of cores 5


# load data

In [4]:
print ("load data set")
data_set = pd.read_hdf('../data_set_.h5',key='DS_300')
print ("*"*80)
# filter data

load data set
********************************************************************************


In [5]:
LE = data_set[(data_set.duration>=effective_days) & (data_set.duration <=max_effective_days)]
print (LE.shape)

(148190, 18)


In [6]:
print("Load long events")
long_events_ = pd.read_hdf('../long_events_.h5',key='LN_300_nc2')
print ("*"*80)
print (long_events_.shape)
long_events_.duration.max()

Load long events
********************************************************************************
(148237, 18)


52435.01666666667

In [7]:
print ("load long_event list")
long_event_dependent_list1 = np.load("../event_list_45_300_1.npy").tolist()
long_event_dependent_list2 = np.load("../event_list_45_300_2.npy").tolist()
long_event_dependent_list = long_event_dependent_list1+long_event_dependent_list2
print (len(long_event_dependent_list))

load long_event list
148237


In [8]:
for x in range(0, cores):
    globals()['dataset%s' % x] = data_set.copy()

# Get list of traffic events within the long event period and summarize it

In [9]:
print("get events within the same time period")
def parjob_event_within_period(data):
    #pick the copy of data set and initilize the performance metrics
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    print("process ",id," started")
    data_frame= globals()['dataset%s' % ((id-1)%cores)]
    counter=0
    size = data.shape[0]
    start=time.time()
    end=0
    
    in_period_df=[]
    group_list=[]
    columns=['RefinedType','counts']
    for idx,long_event in data.iterrows():
        #if long_event.duration <=30*24*60:#max_effective_days:
            index = long_events_.index.get_loc(idx)
            try:
                temp_df = data_frame.loc[long_event_dependent_list[index]]
            except:
                print ("index is ",index)
                print (long_event_dependent_list[index])
                raise
            in_period = temp_df[((temp_df.StartTime >= long_event.StartTime) &
                        (temp_df.EndTime <= long_event.EndTime))]

            collision = in_period[(in_period.duration >= effective_days)& (in_period.duration <= max_effective_days)]
            if not collision.empty:
                grouper = pd.DataFrame(columns=columns).groupby('RefinedType',as_index=False).size().to_frame(name='counts').reset_index()
                group_list.append(grouper)
                #in_period_df.append(pd.DataFrame(columns=columns))
            else:
                grouper = in_period.groupby('RefinedType',as_index=False).size().to_frame(name='counts').reset_index()
                group_list.append(grouper)
                #in_period_df.append(temp_df[((temp_df.StartTime >= long_event.StartTime) &
                #        (temp_df.EndTime <= long_event.EndTime))])

            counter+=1
        #else:
        #    grouper = pd.DataFrame(columns=columns).groupby('RefinedType',as_index=False).size().to_frame(name='counts').reset_index()
        #    group_list.append(grouper)
        
            
            if counter%1000==0:
                end = time.time()
                print(counter,"/",size," long event processes for process ",id,"time=",end-start)
                start = time.time()
            
    #return in_period_df
    return group_list

pool = Pool(cores)
partitions=cores
#related_events_in_period 
count_list_in = applyParallel_list(pool,long_events_,parjob_event_within_period,{})
pool.close()
pool.join()
temp_list=[]
for l in count_list_in:
    temp_list+=l
#related_events_in_period
count_list_in =temp_list.copy()
print ("*"*80)

get events within the same time period
process  1  started
process  2  started
process  3  started
process  4  started
process  5  started
1000 / 29647  long event processes for process  5 time= 7.433563709259033
1000 / 29648  long event processes for process  1 time= 11.92397141456604
1000 / 29647  long event processes for process  3 time= 13.979606866836548
2000 / 29647  long event processes for process  5 time= 10.490427017211914
2000 / 29648  long event processes for process  1 time= 6.381984233856201
2000 / 29647  long event processes for process  3 time= 7.6208655834198
3000 / 29648  long event processes for process  1 time= 4.945322513580322
3000 / 29647  long event processes for process  5 time= 12.0355806350708
1000 / 29648  long event processes for process  2 time= 30.25261402130127
1000 / 29647  long event processes for process  4 time= 30.72899317741394
3000 / 29647  long event processes for process  3 time= 12.080009698867798
4000 / 29647  long event processes for process 

28000 / 29648  long event processes for process  2 time= 7.62313437461853
28000 / 29647  long event processes for process  5 time= 5.280508041381836
17000 / 29647  long event processes for process  4 time= 5.083361625671387
29000 / 29647  long event processes for process  5 time= 4.317281723022461
10000 / 29648  long event processes for process  1 time= 56.996577739715576
18000 / 29647  long event processes for process  4 time= 7.02911901473999
29000 / 29648  long event processes for process  2 time= 13.442283391952515
19000 / 29647  long event processes for process  4 time= 7.259913206100464
20000 / 29647  long event processes for process  4 time= 10.11479640007019
21000 / 29647  long event processes for process  4 time= 15.359302282333374
11000 / 29648  long event processes for process  1 time= 39.86791682243347
22000 / 29647  long event processes for process  4 time= 18.725834846496582
23000 / 29647  long event processes for process  4 time= 22.739108085632324
12000 / 29648  long ev

In [10]:
counter=0
sample_df=count_list_in[0]
for df in count_list_in:
    if not df.empty:
        counter+=1
        sample_df=df
print (counter, " long event list is non_empty out of ",len(count_list_in), " long events")
display(sample_df)

72885  long event list is non_empty out of  148237  long events


,RefinedType,counts
0,Congestion,10


# get events before and after long events (for 1 or more periods) 

In [26]:
print ("Get the events before and after the long event period")
final_data_frame=[]
def parjob_group_out_period(data):
    #get process id and initilize the performance metrics
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    print("process ",id," started")
    data_frame= globals()['dataset%s' % ((id-1)%cores)]
    counter=0
    size = data.shape[0]
    start=time.time()
    end=0
    
    #in_period_df=[]
    out_period_df=[]
    #columns=[ 'RefinedType','out_of_event_avg', 'During_event_avg']
    
    for idx,long_event in data.iterrows():
        #if long_event.duration <=30*24*60:#max_effective_days:
            
            index = long_events_.index.get_loc(idx)
            temp_df = data_frame.loc[long_event_dependent_list[index]]
            event_duration_week_offset = math.ceil(float(long_event.duration)/(7*24*60))

            sp = sample_rate
            i=0
            aggregate_df = pd.DataFrame(columns=['RefinedType'])
            while (sp>0):
                left_temp_df = temp_df[(temp_df.StartTime >= long_event.StartTime-
                                 pd.Timedelta(event_duration_week_offset*7*(i+1), unit='D')) & 
                                (temp_df.EndTime <= long_event.EndTime-
                                  pd.Timedelta(event_duration_week_offset*7*(i+1), unit='D'))
                                 ]
                collision = left_temp_df[(left_temp_df.duration >= effective_days)& (left_temp_df.duration <= max_effective_days)]
                if not collision.empty:
                    pass
                else:
                    grouped = left_temp_df.groupby('RefinedType',as_index=False).size().to_frame(name='Before').reset_index()
                    #display(grouped)
                    #if grouped.empty:
                    #    grouped = pd.DataFrame(columns=['RefinedType','Before'])
                        #in_period_df.append(pd.DataFrame(columns=columns))
                    
                    #if aggregate_df.empty:
                    #    aggregate_df = grouped
                    #else:
                    aggregate_df = pd.merge(aggregate_df, grouped, on='RefinedType', how='outer')
                    #display(aggregate_df)
                    sp-=1
                i+=1
            sp = sample_rate
            i=0
            while (sp>0):
                right_temp_df = temp_df[(temp_df.StartTime >= long_event.StartTime+
                                 pd.Timedelta(event_duration_week_offset*7*(i+1), unit='D')) & 
                                (temp_df.EndTime <= long_event.EndTime+
                                  pd.Timedelta(event_duration_week_offset*7*(i+1), unit='D'))
                                 ]
                collision = right_temp_df[(right_temp_df.duration >= effective_days)& (right_temp_df.duration <= max_effective_days)]
                if not collision.empty:
                    #print ("Collision for long event "+str(idx)+"---> ignore the period "+str(i+1)+ " "+str(collision.shape[0])+" long events overlap")
                    pass
                else:
                    grouped  = right_temp_df.groupby('RefinedType',as_index=False).size().to_frame(name='After').reset_index()
                    #if aggregate_df.empty:
                    #    aggregate_df = grouped
                    #else:
                    aggregate_df = pd.merge(aggregate_df, grouped, on='RefinedType', how='outer')
                    sp-=1
                i+=1
            
            aggregate_df = aggregate_df.fillna(0)
            #out_period_df.append(aggregate_df)
            aggregate_df['mean'] = aggregate_df.mean(axis=1)
            #new_df = aggregate_df[['RefinedType','mean']].copy()
            new_df = aggregate_df.copy()
            new_df.reset_index()
            count_list_in[index].reset_index()
            final_df = pd.merge(count_list_in[index],new_df,on='RefinedType', how='outer')
            #display(final_df)
            col_order = pd.DataFrame(columns=[ 'RefinedType','counts', 'Before','After','mean'])
            final_df = final_df[col_order.columns]
            final_df.columns =[ 'RefinedType','During', 'Before','After','Mean']
                # [ 'RefinedType','During_event_avg','out_of_event_avg']
            final_df = final_df.fillna(0)
            out_period_df.append(final_df)
            
        #else:
        #    out_period_df.append(pd.DataFrame(columns=columns).reset_index())
            
            counter+=1
            if counter%1000==0:
                end = time.time()
                print(counter,"/",size," long event processes for process ",id,"time=",end-start)
                start = time.time()
            
    return out_period_df

pool = Pool(cores)
final_data_frame = applyParallel_list(pool,long_events_,parjob_group_out_period,{})
pool.close()
pool.join()
temp_list=[]
for l in final_data_frame:
    temp_list+=l
final_data_frame=temp_list

print ("*"*80)
print ("*"*80)

Get the events before and after the long event period
process  69  started
process  70  started
process  71  started
process  72  started
process  73  started
1000 / 29647  long event processes for process  73 time= 20.495704650878906
1000 / 29647  long event processes for process  71 time= 25.66020631790161
1000 / 29648  long event processes for process  69 time= 25.81137228012085
1000 / 29647  long event processes for process  72 time= 43.72019648551941
1000 / 29648  long event processes for process  70 time= 44.556151151657104
2000 / 29648  long event processes for process  69 time= 18.9165620803833
2000 / 29647  long event processes for process  73 time= 24.622562646865845
2000 / 29647  long event processes for process  71 time= 20.163299083709717
3000 / 29648  long event processes for process  69 time= 19.68965435028076
2000 / 29647  long event processes for process  72 time= 23.436610460281372
3000 / 29647  long event processes for process  71 time= 27.137885570526123
2000 / 2964

22000 / 29647  long event processes for process  73 time= 22.100001096725464
13000 / 29648  long event processes for process  69 time= 48.26838397979736
25000 / 29648  long event processes for process  70 time= 18.599087953567505
21000 / 29647  long event processes for process  72 time= 32.2353138923645
23000 / 29647  long event processes for process  73 time= 18.259013175964355
26000 / 29648  long event processes for process  70 time= 25.10809588432312
24000 / 29647  long event processes for process  73 time= 20.614238262176514
27000 / 29648  long event processes for process  70 time= 20.2020525932312
22000 / 29647  long event processes for process  72 time= 36.41417694091797
25000 / 29647  long event processes for process  73 time= 16.91592311859131
28000 / 29648  long event processes for process  70 time= 20.526564598083496
26000 / 29647  long event processes for process  73 time= 16.842774391174316
14000 / 29648  long event processes for process  69 time= 69.0089271068573
27000 / 2

In [27]:
index_list=[]
for i in range(0,len(final_data_frame)):
    if final_data_frame[i].empty:
        index_list.append(i)

zero_set = long_events_.iloc[index_list]
zero_set = zero_set[zero_set.duration<24*30*60]
zero_set.to_hdf('../zeroset.h5',key='zero')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [28]:
counter=0
sample_df=final_data_frame[0]
for df in final_data_frame:
    if not df.empty:
        counter+=1
        sample_df=df
print (counter, " long event list is non_empty out of ",len(final_data_frame), " long events")
display(sample_df)

89669  long event list is non_empty out of  148237  long events


,RefinedType,counts,Before,After,mean
0,Congestion,10,2,0.0,1.0


In [34]:
for df in final_data_frame:
    df.columns =[ 'RefinedType','During', 'Before','After','Mean']
display(final_data_frame[300])

,RefinedType,During,Before,After,Mean
0,Congestion,1,0.0,0.0,0.0


In [35]:
import pickle
with open("test.txt", "wb") as fp:
    pickle.dump(final_data_frame, fp)

In [36]:
with open("test.txt", "rb") as fp:
    loaded_final = pickle.load(fp)
#loaded_final

In [32]:
zero_set[(zero_set.Type=='W') & (zero_set.StartTime > '2017-06-01') ]

,EventId,Type,RefinedType_,StartTime,EndTime,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode,RefinedType,duration
954443,W-1311865,W,cold-severe,2017-11-18 00:58:00,2017-11-19 19:58:00,NaN,NaN,NaN,KHEY,NaN,NaN,NaN,Skipperville,NaN,AL,NaN,cold,2580.0
1614976,W-2220411,W,cold-severe,2017-10-09 01:10:00,2017-10-10 07:11:00,NaN,NaN,NaN,KOZR,NaN,NaN,NaN,Slocomb,NaN,AL,NaN,cold,1801.0
1434082,W-1972061,W,rain-moderate,2017-07-05 18:45:00,2017-07-06 18:53:00,NaN,NaN,NaN,KMSL,NaN,NaN,NaN,Florence,NaN,AL,NaN,rain,1448.0
270343,W-371698,W,rain-light,2017-06-20 14:53:00,2017-06-21 13:53:00,NaN,NaN,NaN,KBFM,NaN,NaN,NaN,Mobile,NaN,AL,NaN,rain,1380.0
152806,W-210100,W,rain-light,2017-06-21 14:15:00,2017-06-22 12:55:00,NaN,NaN,NaN,KALX,NaN,NaN,NaN,Sylacauga,NaN,AL,NaN,rain,1360.0
2020474,W-2779083,W,rain-light,2017-06-20 13:40:00,2017-06-21 09:53:00,NaN,NaN,NaN,KTOI,NaN,NaN,NaN,Troy,NaN,AL,NaN,rain,1213.0
13003,W-17733,W,rain-light,2017-06-20 04:58:00,2017-06-20 21:58:00,NaN,NaN,NaN,K0J4,NaN,NaN,NaN,Florala,NaN,AL,NaN,rain,1020.0
1698334,W-2335005,W,rain-light,2017-06-20 18:04:00,2017-06-21 10:56:00,NaN,NaN,NaN,KPRN,NaN,NaN,NaN,HighlandHome,NaN,AL,NaN,rain,1012.0
1348339,W-1854281,W,rain-light,2017-06-20 19:09:00,2017-06-21 11:53:00,NaN,NaN,NaN,KMGM,NaN,NaN,NaN,Selma,NaN,AL,NaN,rain,1004.0
954161,W-1311496,W,cold-severe,2017-07-09 01:58:00,2017-07-09 17:58:00,NaN,NaN,NaN,KHEY,NaN,NaN,NaN,Skipperville,NaN,AL,NaN,cold,960.0


In [33]:
zero_set[zero_set.Type=='T'].groupby('State').size().to_frame('count')

,count
State,
AL,34
AR,25
AZ,22
CA,90
CO,221
CT,31
DE,10
FL,57
GA,125
